### Carbon Python Sample No.02 - GenTab

Contact the Plutonium web service to verify that it is repondoning correctly. The `service/info` endpoint returns environmental and version about the service. Some of the values are printed.

In [7]:
import requests
import json
inforesp = requests.get("https://rcsapps.azurewebsites.net/plutonium/service/info")
#print(inforesp)
infojson = inforesp.json()
#print(json.dumps(infojson, indent=2))
print("Service machine .... " + infojson["hostMachine"])
print("Service version .... " + infojson["version"])
print("Service build ...... " + infojson["build"])
print("Carbon version ..... " + infojson["carbonVersion"])
print("Carbon  build ...... " + infojson["carbonBuild"])


Service machine .... RD0003FF85C098
Service version .... 0.9.33.0
Service build ...... 2022-10-11 16:34 GMT+11
Carbon version ..... 8.3.17.0
Carbon  build ...... 2022-10-08 16:50 GMT+11


Issue a login call with the Carbon guest account credentials to start an authorised session. The response contains detailed information about the account that has created the session. The customers and their child jobs permitted for the account are printed.

In [8]:
postdata={
  "id": "16499372",
  "password": "C6H12O6",
  "skipCache": True
}
loginresp = requests.post("https://rcsapps.azurewebsites.net/plutonium/session/start/login/id", json=postdata)
loginjson = loginresp.json()
sessionId = loginjson["sessionId"]
print("Session ID ........ " + sessionId)
for cust in loginjson["sessionCusts"]:
    print("|  {0} {1}".format(cust["id"], cust["name"]))
    for job in cust["sessionJobs"]:
        print("|  |  {0} {1}".format(job["id"], job["name"]))
sess = requests.Session()
sessheader = { 'x-session-id': sessionId }
sess.headers.update(sessheader)

Session ID ........ E3WPGEYYVC
|  3625340368 rcsruby
|  |  25303847 demo
|  3382340368 rcspublic
|  |  23464980 uk-ge2019
|  |  27811105 cdc-covid
|  |  27468214 firstfleet
|  |  25895325 stockmarkets
|  |  24098249 gss
|  |  23325371 aemo
|  |  22826258 romeo-juliet
|  |  24821945 cov19-global


Open one of the public cloud jobs published by Red Centre Software, seen as the first customer and job entry in the previous response print.

In [9]:
postdata={
  "customerName": "rcsruby",
  "jobName": "demo",
  "getDisplayProps": True,
  "getVartreeNames": True,
  "getAxisTreeNames": True,
  "getTocNew": False,
  "getTocOld": False,
  "getIni": False
}
jobresp = sess.post("https://rcsapps.azurewebsites.net/plutonium/job/open", json=postdata)
jobjson = jobresp.json()
# The job open reponse doesn't contain anything useful for this demonstration.

Generate a simple cross-tabulation report using "Age" and "Region" as the top and side variables respectively. Other service endpoints exist to discover the groups of variables that are defined in a job (not shown in the sample for brevity).

Valid formats are: TSV, CSV, SSV, XLSX, XML, HTML, OXT, Diamond, Pandas, Pandas1, Pandas2. Not all of those are currently useful for Python clients. Research is underway to produce output that is customised for Python consumption.

In [10]:
postdata={
  "name": "Sample02",
  "top": "Age",
  "side": "Region",
  "filter": None,
  "weight": None,
  "format": "CSV",
  "showFrequencies": True,
  "showColumnPercents": False,
  "showRowPercents": False,
  "showStats": False
}
genresp = sess.post("https://rcsapps.azurewebsites.net/plutonium/job/gentab/python", json=postdata)
genjson = genresp.json()
print(json.dumps(genjson, indent=2))

[
  "Name: Sample02",
  "Top: Age",
  "Side: Region",
  ",15-25,26-35,36-50,51-65,65+,Under 35,Over 35,Under 50,Over 50",
  "NE,479,498,501,504,540,977.00,1545.00,1478.00,1044.00",
  "SE,459,441,510,514,535,900.00,1559.00,1410.00,1049.00",
  "SW,523,467,457,517,511,990.00,1485.00,1447.00,1028.00",
  "NW,469,510,542,499,524,979.00,1565.00,1521.00,1023.00",
  "East,938,939,1011,1018,1075,1877.00,3104.00,2888.00,2093.00",
  "West,992,977,999,1016,1035,1969.00,3050.00,2968.00,2051.00",
  "North,948,1008,1043,1003,1064,1956.00,3110.00,2999.00,2067.00",
  "South,982,908,967,1031,1046,1890.00,3044.00,2857.00,2077.00"
]


When job processing is complete, the job should be closed to immediately free resources. The response contains the boolean value `True` or `False` to indicate if the job was closed.

In [11]:
closeresp = sess.delete("https://rcsapps.azurewebsites.net/plutonium/job/close")
closejson = closeresp.json()
print(json.dumps(closejson, indent=2))

true


Issue a 'logoff' request. The session will be ended and the usage count for the account is decremented. The integer return value is the new licence usage count for the account.

In [12]:
endresp = sess.delete("https://rcsapps.azurewebsites.net/plutonium/session/end/logoff")
endjson = endresp.json()
print(json.dumps(endjson, indent=2))
sess.close()

1
